# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [52]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# import libraries
import re
import pandas as pd

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(engine.table_names()[0],con=engine)
X = df.message
y = df.iloc[:,5:]

In [79]:
category_names = y.columns

In [19]:
for col in y.columns:
    y.loc[y[col] > 1, col] = 1

### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize sentence
    token = word_tokenize(text)
    # remove stopwords and lemmatize 
    words = [WordNetLemmatizer().lemmatize(w) for w in token if w not in stopwords.words('english')]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [25]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)
print(classification_report(y_test, Y_pred, target_names = category_names))
print('---------------------------------')

In [38]:
def performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print('Label: {}'.format(col))
        print('------------------------------------------------------')
        print(classification_report(y_test[col], y_pred[:, i]))

performance(pipeline, X_test, y_test)

Label: related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.63      0.44      0.52      1572
          1       0.84      0.92      0.88      4982

avg / total       0.79      0.80      0.79      6554

Label: request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5400
          1       0.79      0.44      0.56      1154

avg / total       0.87      0.88      0.87      6554

Label: offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

Label: aid_related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:
y_pred_base = pipeline.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred_base))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred_base, average='weighted')))

Testing accuracy 0.251296917913
Testing F1 score: 0.5489957093801142


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
# Check the list of available parameters with `estimator.get_params().keys()`.
list(pipeline.get_params().keys())

['memory',
 'steps',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimat

In [29]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range':[(1,2),(2,2)],
              
              'clf__estimator__n_estimators':[10, 30, 50]}

gs_clf = GridSearchCV(estimator = pipeline, param_grid=parameters)

In [30]:
gs_clf.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': [(1, 2), (2, 2)], 'clf__estimator__n_estimators': [10, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [67]:
y_pred_gs = gs_clf.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred_gs))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred_gs, average='weighted')))

Testing accuracy 0.269453768691
Testing F1 score: 0.5460617323784557


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [33]:
best_parameters = gs_clf.best_params_
best_accuracy = gs_clf.best_score_
print(best_parameters)
print(best_accuracy)

{'clf__estimator__n_estimators': 50, 'vect__ngram_range': (1, 2)}
0.265079849456


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
y_pred = gs_clf.predict(X_test)

In [63]:
from sklearn.metrics import accuracy_score,f1_score

In [39]:
performance(gs_clf, X_test, y_test)

Label: related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.68      0.44      0.53      1572
          1       0.84      0.93      0.89      4982

avg / total       0.80      0.82      0.80      6554

Label: request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5400
          1       0.83      0.48      0.60      1154

avg / total       0.89      0.89      0.88      6554

Label: offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

Label: aid_related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


- After tune parameters of the model using grid search. The precision, recall, and f1-score to predict each label as 1 are much better than the base model previously.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [48]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import AdaBoostClassifier

In [55]:
def new_model_pipeline():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [56]:
new_model_pipeline = new_model_pipeline()

In [57]:
new_model_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [58]:
new_model_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [59]:
performance(new_model_pipeline, X_test, y_test)

Label: related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.69      0.20      0.31      1572
          1       0.79      0.97      0.87      4982

avg / total       0.77      0.79      0.74      6554

Label: request
------------------------------------------------------
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      5400
          1       0.74      0.50      0.60      1154

avg / total       0.87      0.88      0.87      6554

Label: offer
------------------------------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      0.99      0.99      6554

Label: aid_related
------------------------------------------------------
             precision    recall  f1-score   support

          0       0

### 9. Export your model as a pickle file

In [92]:
from sklearn.externals import joblib

In [94]:
joblib.dump(gs_clf, 'classification.pkl')


['classification.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.